In [1]:
%load_ext autoreload
%autoreload

import os
import numpy as np
import cv2
from osgeo import gdal
import matplotlib.pyplot as plt
import glob

import sys
sys.path.append('/home/golden/coding/drone-tracking/code/functions')
import mapping_functions as kmap
import koger_general_functions as kgf

from drone_movement import create_gt_segment_dicts, get_segment_drone_movement
from drone_movement import get_warp

In [2]:
observation_name = 'observation088'
# observation_name = 'observation076'

plot_save_folder = "/home/golden/Dropbox/drone-paper/figures/supplement"

kenya_drones_path = "/home/golden/kenya_drones"
maps_folder = os.path.join(kenya_drones_path,
                           "3D_mapping/small_maps"
                          )

tracks_file = os.path.join(kenya_drones_path, 
                       "processing/kenya-tracking/processed-videos/raw-footage",
                       observation_name, 
                       "localizations/tracks-complete.npy"
                      )

In [3]:
frame_folders_root = os.path.join(kenya_drones_path, 
                                  "processing/kenya-tracking/raw-frames/raw-footage",
                                  observation_name
                                 )
pix4d_folder = os.path.join(maps_folder, observation_name)

pmatrix_file = pix4d_folder + '/1_initial/params/' + observation_name + '_pmatrix.txt' 
pmatrix_list = kmap.create_pmatrix_dicts(pmatrix_file)

# Load the saved flight logs that contain both drone sensor info and the video frame info
flight_logs_folder = os.path.join(pix4d_folder, 'drone-logs')
flight_logs = kmap.get_cleaned_flight_logs(flight_logs_folder,
                                          pmatrix_list
                                          )
segment_dicts = create_gt_segment_dicts(frame_folders_root, 
                                        flight_logs, 
                                        save=False)
gt_obs_inds, gt_frame_names = kmap.get_groundtruth_obs_indexes(flight_logs, 
                                                     frame_folders_root, 
                                                     for_test=False, use_old_method=False
                                                    )

2 gt images not used in final map.
APR08_2018_A_DJI_0008_08226.jpg not found
trying APR08_2018_A_DJI_0008_08224.jpg
gt not found. using APR08_2018_A_DJI_0008_08224.jpg for APR08_2018_A_DJI_0008_08226.jpg.
APR08_2018_A_DJI_0008_08226.jpg not found
trying APR08_2018_A_DJI_0008_08224.jpg
gt not found. using APR08_2018_A_DJI_0008_08224.jpg for APR08_2018_A_DJI_0008_08226.jpg.


In [5]:
x = 10
f"d{x:04d}"

'd0010'

In [4]:
gt_frame_names[32:35]

['/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_A_DJI_0008/APR08_2018_A_DJI_0008_07128.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_A_DJI_0008/APR08_2018_A_DJI_0008_08224.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_B_DJI_0005/APR08_2018_B_DJI_0005_07628.jpg']

In [15]:
filenames[30439:30444]

['/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_A_DJI_0008/APR08_2018_A_DJI_0008_08222.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_A_DJI_0008/APR08_2018_A_DJI_0008_08224.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_B_DJI_0005/APR08_2018_B_DJI_0005_07624.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_B_DJI_0005/APR08_2018_B_DJI_0005_07626.jpg',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation088/APR08_2018_B_DJI_0005/APR08_2018_B_DJI_0005_07628.jpg']

In [4]:
gt_names = []
for fl in flight_logs:
    gt_inds = fl['used_in_map']
    gt_names.extend(fl.loc[gt_inds, 'image_name'].to_list())

In [5]:
gt_names[32:35]

['APR08_2018_A_DJI_0008_07128.jpg',
 'APR08_2018_A_DJI_0008_08226.jpg',
 'APR08_2018_B_DJI_0005_07628.jpg']

In [57]:
gt_frame_names[:1]

['/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation047/NOV14_2017_B_DJI_0063/NOV14_2017_B_DJI_0063_06568.jpg']

In [12]:
filenames = kgf.get_observation_frame_files(frame_folders_root)

In [59]:
log_ind = 0
fl = flight_logs[log_ind]
gt_names = fl.loc[fl['used_in_map']==True, 'image_name'].to_list()


In [60]:
alt_gt_obs_inds = []

gt_names = []


for fl in flight_logs:
    gt_names.extend(fl.loc[fl['used_in_map']==True, 'image_name'].to_list())
    
last_gt_obs_ind = 0

for gt_ind, name in enumerate(gt_names):
    for obs_ind, f in enumerate(filenames[last_gt_obs_ind:]):
        found = False
        if name in f:
            alt_gt_obs_inds.append(obs_ind + last_gt_obs_ind)
            last_gt_obs_ind += obs_ind
            found = True
            break
            
    if not found:
        print(f"gt not found. setting to last frame. gt ind: {gt_ind}")
        alt_gt_obs_inds.append(len(filenames)-1)
        

In [61]:
alt_gt_obs_inds[:10]

[2, 65, 146, 224, 539, 653, 686, 917, 989, 1013]

In [64]:
alt_gt_obs_inds[300:320], gt_obs_inds[300:320]

([50488,
  50509,
  50530,
  50548,
  50569,
  50590,
  50620,
  50629,
  50665,
  50677,
  50698,
  50716,
  50731,
  50746,
  50761,
  50773,
  50797,
  50848,
  50857,
  50875],
 [50487,
  50508,
  50529,
  50547,
  50568,
  50589,
  50619,
  50628,
  50664,
  50676,
  50697,
  50715,
  50730,
  50745,
  50760,
  50772,
  50796,
  50847,
  50856,
  50874])

In [29]:
glob_string = f"/home/golden/kenya_drones/3D_mapping/small_maps/{observation_name}/map-images/*.jpg"
gt_image_files = sorted(glob.glob(glob_string))

In [30]:
max_gt_ind = 10

all_files = [os.path.basename(filenames[obs_ind]) for obs_ind in gt_obs_inds[:max_gt_ind]]
gt_files = [os.path.basename(f) for f in gt_image_files[:max_gt_ind]]

In [31]:
print(*all_files, sep='\n')
print('*'*5)
print(*gt_files, sep='\n')

NOV14_2017_B_DJI_0063_06568.jpg
NOV14_2017_B_DJI_0063_06694.jpg
NOV14_2017_B_DJI_0063_06856.jpg
NOV14_2017_B_DJI_0063_07012.jpg
NOV14_2017_B_DJI_0063_07642.jpg
NOV14_2017_B_DJI_0063_07870.jpg
NOV14_2017_B_DJI_0063_07936.jpg
NOV14_2017_B_DJI_0063_08398.jpg
NOV14_2017_B_DJI_0063_08542.jpg
NOV14_2017_B_DJI_0063_08590.jpg
*****
NOV14_2017_B_DJI_0063_06572.jpg
NOV14_2017_B_DJI_0063_06698.jpg
NOV14_2017_B_DJI_0063_06860.jpg
NOV14_2017_B_DJI_0063_07016.jpg
NOV14_2017_B_DJI_0063_07646.jpg
NOV14_2017_B_DJI_0063_07874.jpg
NOV14_2017_B_DJI_0063_07940.jpg
NOV14_2017_B_DJI_0063_08402.jpg
NOV14_2017_B_DJI_0063_08546.jpg
NOV14_2017_B_DJI_0063_08594.jpg


In [32]:
log_ind = 0
fl = flight_logs[log_ind]
gt_names = fl.loc[fl['used_in_map']==True, 'image_name'].values

In [44]:
flight_ind = 0
list(flight_logs[flight_ind].loc[flight_logs[flight_ind]['observing'], 'frame_num'])[0]


6572

In [38]:
fl.loc[1067:]

,latitude,longitude,altitude(feet),ascent(feet),speed(mph),distance(feet),time(millisecond),datetime(utc),satellites,voltage(v),...,voltageCell6,battery_temperature(f),flycStateRaw,flycState,message,frame_num,altitude(m),observing,image_name,used_in_map
1067,0.354920,36.898893,6135.566567,326.771664,0.315409,988.228378,107700,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6560,1870.599563,False,None,False
1068,0.354920,36.898893,6135.566567,326.771664,0.315409,988.169323,107800,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6566,1870.599563,False,None,False
1069,0.354920,36.898893,6135.566567,326.771664,0.315409,988.129953,107900,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6572,1870.599563,True,NOV14_2017_B_DJI_0063_06572.jpg,True
1070,0.354920,36.898893,6135.894651,327.099748,0.315409,988.087302,108000,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6578,1870.699589,True,None,False
1071,0.354920,36.898893,6135.894651,327.099748,0.223694,988.067617,108100,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6584,1870.699589,True,None,False
1072,0.354920,36.898892,6135.894651,327.099748,0.223694,988.041370,108200,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6590,1870.699589,True,None,False
1073,0.354920,36.898892,6135.894651,327.099748,0.223694,988.031528,108300,2017-11-14 07:30:25,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6596,1870.699589,True,None,False
1074,0.354920,36.898892,6135.894651,327.099748,0.223694,988.015123,108400,2017-11-14 07:30:26,19,16.427,...,0,82.31,6,GPS_Atti,NaN,6602,1870.699589,True,None,False
1075,0.354920,36.898892,6135.894651,327.099748,0.223694,988.011843,108500,2017-11-14 07:30:26,19,16.320,...,0,82.31,6,GPS_Atti,NaN,6608,1870.699589,True,None,False
1076,0.354920,36.898892,6135.894651,327.099748,0.223694,988.008562,108600,2017-11-14 07:30:26,19,16.320,...,0,82.31,6,GPS_Atti,NaN,6614,1870.699589,True,None,False
